In [1]:
import numpy as np
import re
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from pickle import dump
from pickle import load 
from keras.models import load_model
import pickle

Using TensorFlow backend.


In [2]:
# Load model and mapping
model = load_model("model5_6.h5")
mapping = load(open("mapping.pkl", "rb"))
vocab_size = len(mapping)

In [3]:
# Sample from predictions (array a) with temperature as hyperparameter
# Temp = 1 as default -> normal unscaled 
# Returns the integer index corresponding to the predicted character
def sample(a, temperature=1.0):
    a = np.asarray(a).astype('float64')
    a = a**(1/temperature)
    p_sum = a.sum()
    sample_temp = a/p_sum 
    return np.argmax(np.random.multinomial(1, sample_temp, 1))

In [4]:
def character(text): 
    # Convert all character except newline (not in dictionary) to integers
    encoded_seed = [mapping[char] for char in text if char != "\n"]
    # Pad sequences to be of fixed length 40
    encoded_seed = pad_sequences([encoded_seed], maxlen=40, truncating='pre')
    # Return hot one-encoded 
    return np.array(to_categorical(encoded_seed, num_classes = vocab_size))

In [7]:
# Fixed 40 character per line, 14 lines poem generation 
seed = "Shall I compare thee to a summer's day?\n"
# Number of current lines
num_line = 1
# Keep track of text of poem
text = seed
# Current character count of line 
curr_line_len = 0
# Hard cutoff
curr_line_cutoff = 40

# Temperature hyperparameter 
temp = 0.5

while True: 
    # If current line 40 characters, line break
    if curr_line_len == curr_line_cutoff:
        num_line += 1
        # If we have 14 lines, finish poem generation
        if num_line == 14: 
            break
        text += "\n"
        curr_line_len = 0
    
    encoded_seed = character(text)
    # Get array of predictions from model 
    predict = model.predict(encoded_seed, verbose=0)[0]
    next_index = sample(predict, temp)
    
    # Find the character corresponding to the integer index
    for char, index in mapping.items(): 
        if index == next_index: 
            out_char = char 
            break 
    # Add character to text
    text += out_char
    # Add to current length of line
    curr_line_len += 1

In [8]:
print(text)

Shall I compare thee to a summer's day?
As enough thou thy dear heart that perju
red with the state,It thy self and such 
would be the deart,If thy self the leeve
st of the shore, black,Had the store the
 thoughts of the despise,Whill I compoun
ted and love with false,But from thy bea
uty with thy self art thee.As thy heart 
in thy hue, for I may shall refore,As I 
shall be thy sourly memory:And my love t
hat love that mine eye doth decay,To thi
nk in my love that I have spent.Like of 
the summer's water by thy self too short
,Nor thy scythe tongues that the state b


In [23]:
# Smarter poem generation (no fixed line length)

seed = "shall i compare thee to a summer's day?\n"
# Number of current lines
num_line = 1
# Keep track of text of poem
text = seed
# Current character count of line 
curr_line_len = 0
curr_line_threshold = 30

# Temperature hyperparameter 
temp = 0.5

while True:  
    encoded_seed = character(text)
    # Get array of predictions from model 
    predict = model.predict(encoded_seed, verbose=0)[0]
    next_index = sample(predict, temp)
    
    # Find the character corresponding to the integer index
    for char, index in mapping.items(): 
        if index == next_index: 
            out_char = char 
            break 
    
    # Once above threshold for number of characters in line
    # Each capital letter indicate newline 
    if curr_line_len >= curr_line_threshold and out_char.istitle(): 
        num_line += 1
        # If we have 14 lines, finish poem generation
        if num_line == 14: 
            break
        text += "\n"
        curr_line_len = 0
    # Add character to text
    text += out_char
    # Add to current length of line
    curr_line_len += 1
        

In [24]:
print(text)

shall i compare thee to a summer's day?
Why then thou beauty's store what a count,
And play is my heart's great quick,
To seek from the would have bright in thee,
And thou art all thy heart in outwards
A makener thoughts canst not thou shalt waste 
I be.But sweet hanged in the liver with men's thee.
For you wide, when thou mayst thou bear thy shade, 
I see love speak,To true with self-rest the world with count, of bears,
That is heaven the self-deserving still,
To the conceit of the time absent with spent.
O let seem not so, I love the store thee.
So then I see thee the perce that beauty?
Or which thou thy self to the change still,
